In [ ]:
from pathlib import Path
import os, shutil
import zipfile
import tarfile
import glob

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import logging

import memory_profiler
%load_ext memory_profiler
import gc
import cartopy.crs as ccrs

from s2driver import driver_S2_SAFE as S2, __version__

from s2cloudless import S2PixelCloudDetector, download_bands_and_valid_data_mask

from grs import product

import grstbx
from grstbx import visual
ust = grstbx.utils.spatiotemp()
opj = os.path.join
__version__

In [ ]:
bandIds = range(13)
resolution = 20
datadir = '/data/satellite/Sentinel-2/L1C/'
odir = '/data/satellite/Sentinel-2/projet/robert/L1C'
tile = '30PYT'
clon,clat = -0.66,11.6
width, height = 35000, 35000
complevel=5

In [ ]:
year='2021'
files = glob.glob(opj(datadir,tile,year,'*','*','*N05*SAFE'))

In [ ]:
ust = grstbx.utils.spatiotemp()

for file in files:

    box = ust.wktbox(clon, clat, width=width, height=height, ellps='WGS84')
    bbox = gpd.GeoSeries.from_wkt([box]).set_crs(epsg=4326)
    subset = bbox
    
   
    file_nc = os.path.basename(file).replace('.SAFE', '.nc')
    file_nc = opj(odir,year,file_nc)
    odir_ = os.path.dirname(file_nc)
    
    if not os.path.exists(file_nc):
        print(file_nc)
    #if True:
        l1c= S2.sentinel2_driver(file, band_idx=bandIds,subset=subset, resolution_angle=60,resolution=resolution)
        l1c.load_product()
        prod = product(l1c.prod)
        encoding = {'bands': {'dtype': 'int16', 'scale_factor': 0.000022, 'add_offset': .3, '_FillValue': -32768,"zlib": True,
                                           "complevel": complevel, 'grid_mapping': 'spatial_ref'},
                    'vza': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999,"zlib": True,
                                           "complevel": complevel, 'grid_mapping': 'spatial_ref'},
                    'raa': {'dtype': 'int16', 'scale_factor': 0.1, '_FillValue': -9999,"zlib": True,
                                           "complevel": complevel, 'grid_mapping': 'spatial_ref'},
                    'sza': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999,"zlib": True,
                                           "complevel": complevel, 'grid_mapping': 'spatial_ref'}
                    }
        
        # apply cloud mask
        bands = np.array([l1c.prod.bands.values.transpose((1,2,0))])
        
        cloud_detector = S2PixelCloudDetector(threshold=0.6, average_over=1, dilation_size=8, all_bands=True)
        probability_maps = cloud_detector.get_cloud_probability_maps(bands)
        cloud_mask = cloud_detector.get_mask_from_prob(probability_maps)
        l1c.prod['cloud_p06'] = (('y','x'),cloud_mask[0])
        cloud_detector = S2PixelCloudDetector(threshold=0.8, average_over=1, dilation_size=5, all_bands=True)
        cloud_mask = cloud_detector.get_mask_from_prob(probability_maps)
        l1c.prod['cloud_p08'] = (('y','x'),cloud_mask[0])
        encoding['cloud_p06']={"zlib": True, "complevel": complevel, 'grid_mapping': 'spatial_ref'}
        encoding['cloud_p08']={"zlib": True, "complevel": complevel, 'grid_mapping': 'spatial_ref'}
        
        if not os.path.exists(odir_):
            os.makedirs(odir_)
        l1c.prod.to_netcdf(file_nc, encoding=encoding)
        l1c.prod.close()
        del l1c
        gc.collect()

In [ ]:


file = '/data/satellite/Sentinel-2/L1C/30PYT/2021/05/11/S2B_MSIL1C_20210511T101559_N0500_R065_T30PYT_20230207T184732.SAFE'

box = ust.wktbox(clon, clat, width=width, height=height, ellps='WGS84')
bbox = gpd.GeoSeries.from_wkt([box]).set_crs(epsg=4326)
subset = bbox


file_nc = os.path.basename(file).replace('.SAFE', '.nc')
file_nc = opj(odir,year,file_nc)
odir_ = os.path.dirname(file_nc)


l1c= S2.sentinel2_driver(file, band_idx=bandIds,subset=subset, resolution_angle=60,resolution=10)
l1c.load_product()
prod = product(l1c.prod)
encoding = {'bands': {'dtype': 'int16', 'scale_factor': 0.000022, 'add_offset': .3, '_FillValue': -32768,"zlib": True,
                                   "complevel": complevel, 'grid_mapping': 'spatial_ref'},
            'vza': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999,"zlib": True,
                                   "complevel": complevel, 'grid_mapping': 'spatial_ref'},
            'raa': {'dtype': 'int16', 'scale_factor': 0.1, '_FillValue': -9999,"zlib": True,
                                   "complevel": complevel, 'grid_mapping': 'spatial_ref'},
            'sza': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999,"zlib": True,
                                   "complevel": complevel, 'grid_mapping': 'spatial_ref'}
            }

In [ ]:
# apply cloud mask
bands = np.array([l1c.prod.bands.values.transpose((1,2,0))])

cloud_detector = S2PixelCloudDetector(threshold=0.6, average_over=1, dilation_size=8, all_bands=True)
probability_maps = cloud_detector.get_cloud_probability_maps(bands)
cloud_mask = cloud_detector.get_mask_from_prob(probability_maps)
l1c.prod['cloud_p06'] = (('y','x'),cloud_mask[0])
cloud_detector = S2PixelCloudDetector(threshold=0.8, average_over=1, dilation_size=5, all_bands=True)
cloud_mask = cloud_detector.get_mask_from_prob(probability_maps)
l1c.prod['cloud_p08'] = (('y','x'),cloud_mask[0])
encoding['cloud_p06']={"zlib": True, "complevel": complevel, 'grid_mapping': 'spatial_ref'}
encoding['cloud_p08']={"zlib": True, "complevel": complevel, 'grid_mapping': 'spatial_ref'}


if not os.path.exists(odir_):
    os.makedirs(odir_)
l1c.prod.to_netcdf(file_nc, encoding=encoding)
#l1c.prod.close()
#del l1c
#gc.collect()

In [ ]:

prod = product(l1c.prod)

In [ ]:
v=visual.view_spectral(prod.raster.bands,reproject=True)
v.title="## S2 L1C"
v.minmaxvalues=(0,0.3)
v.minmax=[0,0.5]
v.visu()

In [ ]:
geom_ = v.get_geom(v.aoi_stream,crs=prod.raster.rio.crs)

In [ ]:

import cartopy.crs as ccrs
str_epsg = str(l1c.prod.rio.crs)
zone = str_epsg[-2:]
is_south = str_epsg[2] == 7
proj = ccrs.UTM(zone, is_south)
plt.figure(figsize=(25,25))
l1c.prod.bands.sel(wl=[665,560,490]).plot.imshow(rgb='wl', robust=True,subplot_kws=dict(projection=proj))